# Practica Sistemas distribuidos: MotoGP Qatar 2014

## Enunciados de la sede de Argüelles

Los ejercicios se han resuelto usando DataFrames pero algunos también usando spark.sql, para hacer la comparación y aprender.

In [1]:
#conexion a spark
import pyspark   
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("understanding_sparksession")
    .getOrCreate() )
sc = spark.sparkContext
print(spark)
print(sc)

<SparkContext master=local[*] appName=understanding_sparksession>


In [2]:
#Definición del esquema del fichero:
from pyspark.sql.types import *
from pyspark.sql.functions import *
customSchema = StructType([StructField("Id", LongType(), True),
                           StructField("Parent_sys_id", StringType(), True),
                           StructField("Source", StringType(), True),
                           StructField("Mentions", StringType(), True),
                           StructField("Target", StringType(), True),
                           StructField("Name_source", StringType(), True),
                           StructField("Body", StringType(), True),
                           StructField("Pub_date", TimestampType(), True),
                           StructField("URLs", StringType(), True),
                           StructField("Tipe_action", StringType(), True),
                           StructField("Link", StringType(), True),
                           StructField("Has_link", ByteType(), True),
                           StructField("Has_picture", ByteType(), True),
                           StructField("Website", StringType(), True),
                           StructField("Country", StringType(), True),
                           StructField("Activity", LongType(), True),
                           StructField("Followers", LongType(), True),
                           StructField("Following", LongType(), True),
                           StructField("Location", StringType(), True)
                          ])

In [3]:
#lectura fichero
events = spark.read.csv("DATASET-Twitter-23-26-Mar-2014-MotoGP-Qatar.csv",
                    header=True, schema=customSchema, timestampFormat="dd/MM/yyyy HH:mm")
events.count()

257680

In [4]:
#creacion de la colección tweets para los casos en que se usa spark.sql
events.createOrReplaceTempView('tweets')

# Ejercicio 2

## a) Calcular el número total de retweets por usuario para los 50 usuarios con más mensajes en la muestra de tweets analizados. 

In [5]:
#50 ususarios con mas mensajes ordenados de mas a menos
a=(events
 .groupBy("Source")
 .agg(count("Id").alias("tweets"))
 .orderBy("tweets", ascending=False)
 .limit(50))
 
#filtrado de mensajes a los tweets que son retweets, tipo: RT
b=(events.select("Source","Id","Tipe_action")
.filter(events.Tipe_action == 'RT')
.groupBy("Source")
.agg(count("Id").alias("retweets"))
 )

# Union, objeter b solo para la lista de usuarios (Source) de a

c=b.join(a,"Source",how='right')
c.orderBy("tweets", ascending=False).show(50)



+---------------+--------+------+
|         Source|retweets|tweets|
+---------------+--------+------+
|     m_azharaji|       1|   486|
|     twitMOTOGP|     401|   401|
|      johnbokke|     289|   297|
|   qatarflights|     215|   283|
|     box_repsol|      48|   267|
|    yolandaa_95|     126|   185|
|    AlessiaPont|     181|   182|
|     MM93Lovers|     169|   169|
|    motomatters|      39|   169|
|     Sonic_Moto|      94|   165|
|  noelia_260797|     128|   157|
|    birtymotogp|       2|   155|
|  trackseven707|     152|   152|
|   crash_motogp|       1|   149|
|MarcMarquezTeam|      52|   148|
|    pedrosistas|      24|   148|
|   MotoFamilyGP|      17|   144|
|   Kay46_MotoGP|      54|   142|
|    blogenboxes|      15|   133|
|     Ciintiia93|     119|   133|
|      JesiPacio|     118|   130|
|     tigrescuba|     123|   125|
|    VAVEL_motor|    null|   124|
|   MotoGPquotes|     123|   123|
|     thalia_26_|      60|   119|
|       plusmoto|    null|   119|
|   iNotaMenta

## Calcular, para cada uno de estos usuarios la media de enlaces (URLs) enviados por mensaje. (2.5 puntos).¶

In [6]:
#50 ususarios con mas mensajes ordenados de mas a menos
a=(events
 .groupBy("Source")
 .agg(count("Id").alias("tweets"))
 .orderBy("tweets", ascending=False)
 .limit(50))

#separacion de los datos en el campo URLs, que contiene separado por comas los URLs 
#de cada mensaje

aa=(events.select("Source","Id","URLs")
.select("Source","Id",split("URLs",',').alias("uris"))
.select("Source","Id",explode("uris").alias("luris")))

#Media de los URLs por mensaje para cada usuario
bb=(aa
.groupBy("Source","Id")
.agg(count("luris").alias("nuris"))
.groupBy("Source")
.agg(mean("nuris").alias("med_uris")))

#Media de los URLs por mensaje para cada usuario para los 50 usuarios 
#con mas mensajes (a) ordenados por el valor de la media de URLs que usan
fff = bb.join(a,"Source",how='right')
fff.orderBy("med_uris",ascending=False).show(50)

+---------------+------------------+------+
|         Source|          med_uris|tweets|
+---------------+------------------+------+
|      AsyrafAye|1.2352941176470589|   102|
|     tigrescuba|             1.224|   125|
|    VAVEL_motor|1.1612903225806452|   124|
|   MotoGPquotes|1.1219512195121952|   123|
|  trackseven707|1.0986842105263157|   152|
|     Ciintiia93|1.0827067669172932|   133|
|     twitMOTOGP|1.0822942643391522|   401|
|        salo_98| 1.079646017699115|   113|
|   rohimahfidia|1.0761904761904761|   105|
|  btsportmotogp|1.0707964601769913|   113|
|  noelia_260797| 1.070063694267516|   157|
|      _montse24| 1.065217391304348|    92|
|   qatarflights|1.0600706713780919|   283|
|    Carlota_147| 1.055045871559633|   109|
|      JesiPacio|1.0538461538461539|   130|
|   crash_motogp|1.0536912751677852|   149|
|nisaauliarahma5|1.0495049504950495|   101|
|    pedrosistas|1.0472972972972974|   148|
|      johnbokke|1.0437710437710437|   297|
|         MotoGP|1.0421052631578

## b) Calcular el número total de mensajes que contienen información de geolocalización en el campo LOCATION. (2.5 puntos).

In [8]:
#Aunque en la información se dice que el campo de LOCATION si comienza por “ÜT”, se proporcionan
#las coordenadas exactas desde donde se emitió el tweet, hay tweets que también dan la localización
#y que empiezan por un espacio seguido de “ÜT”, y otros que empiezan por "UT". Los demás o 
#muestran la localización dada por el usuario, lugar y no coordenadas o no tienen nada.
#En mi opinión deberían sumarse todos los que contienen coordenadas.

#Número de mensajes con geolocalización si nos restringimos a la información:

events.filter(events.Location.startswith('ÜT')).count()

2136

In [9]:
#Con spark.sql
numloc= spark.sql("""
SELECT Location FROM tweets
WHERE Location LIKE 'ÜT%'
""")
numloc.count()

2136

In [10]:
#Número de mensajes con geolocalización si tomamos todos: 'ÜT', ' ÜT' y 'UT'

events[events.Location.contains('ÜT') | events.Location.contains('UT')].count()


2333

In [11]:
#Con spark.sql
numloc= spark.sql("""
SELECT Location FROM tweets
WHERE Location LIKE '%ÜT%' OR Location LIKE '%UT%'
""")
numloc.count()

2333

## c) Calcular las 10 cuentas de Twitter que más han sido mencionadas en todo el conjunto de datos analizados. (2.5 puntos).

In [12]:
#Los tweets con menciones seran tweets retweet o con mencion "RT" o "MT". Como el campo
#Menciones lista varios nicks, se separan primero

#Separacion de las cuentas mencionadas en el campo Mentions
aaa=(events.select("Tipe_action","Mentions")
.select("Tipe_action",split("Mentions",',').alias("men"))
.select("Tipe_action",explode("men").alias("mens")))
    
#filtrado a retweets y tweets con mention 
bbb=(aaa
.filter((events.Tipe_action == "RT") | (events.Tipe_action == "MT"))  
.groupBy("mens")
.agg(count("mens").alias("Menciones"))
.orderBy("Menciones", ascending=False)
.limit(10)
.show())



+--------------+---------+
|          mens|Menciones|
+--------------+---------+
|  valeyellow46|    61226|
| marcmarquez93|    58132|
|        motogp|    38219|
|     lorenzo99|    16802|
|26_danipedrosa|    12346|
| alexmarquez23|     6117|
|    box_repsol|     5304|
|     jessansan|     4568|
|           ims|     3325|
| btsportmotogp|     2479|
+--------------+---------+



## d) Calcular los 10 mensajes más retweeteados y los 10 mensajes que han acumulado más respuestas en la muestra de datos analizados. 

In [13]:
#10 mensajes mas retweeteados: Mensajes de typo RT y como los que tienen padre=sin padre no 
#estan entre los 10 primeros se pueden poner o no. Se ha impuesto solo en la version con sql. 


(events.filter(events.Tipe_action == "RT")
 .groupBy("Parent_sys_id","Body")
.agg(count("id").alias("tweets"))
.orderBy("tweets", ascending=False)
.limit(10)
 .show())


+-------------+--------------------+------+
|Parent_sys_id|                Body|tweets|
+-------------+--------------------+------+
|       603804|rt @marcmarquez93...|  5510|
|       674825|rt @valeyellow46:...|  4993|
|       460820|rt @marcmarquez93...|  3320|
|       758581|rt @marcmarquez93...|  3235|
|       553100|rt @motogp: #qata...|  2873|
|       681159|rt @valeyellow46:...|  2405|
|       553344|rt @motogp: anoth...|  2264|
|       666385|rt @marcmarquez93...|  1773|
|       605727|rt @lorenzo99: pe...|  1690|
|       488086|rt @motogp: for t...|  1640|
+-------------+--------------------+------+



In [14]:
#10 mensajes mas retweeteados:
#Con spark.sql

masret=spark.sql("""
SELECT Parent_sys_id, Body, count(Id) as `tweets` FROM tweets
WHERE Tipe_action = "RT"  and Parent_sys_id !="sin padre"
GROUP BY  Parent_sys_id, Body
ORDER BY tweets DESC
LIMIT 10
""")
masret.show()



+-------------+--------------------+------+
|Parent_sys_id|                Body|tweets|
+-------------+--------------------+------+
|       603804|rt @marcmarquez93...|  5510|
|       674825|rt @valeyellow46:...|  4993|
|       460820|rt @marcmarquez93...|  3320|
|       758581|rt @marcmarquez93...|  3235|
|       553100|rt @motogp: #qata...|  2873|
|       681159|rt @valeyellow46:...|  2405|
|       553344|rt @motogp: anoth...|  2264|
|       666385|rt @marcmarquez93...|  1773|
|       605727|rt @lorenzo99: pe...|  1690|
|       488086|rt @motogp: for t...|  1640|
+-------------+--------------------+------+



In [15]:
#10 mensajes con mas respuestas: Al ser respuestas deben ser originales TW o con menciones y se
#impone que tengan Padre (distinto de sin padre) porque hay muchos sin, y el contenido no nulo 
#(Body not null). Hay bastantes respuestas sin contenido pero opino que no tiene sentido
#ponerlas y por eso hago la restricion de BODY not null.

(events[(events.Tipe_action == "TW") | (events.Tipe_action == "MT")]
 .filter(events.Parent_sys_id != "sin padre")
 .filter(events.Body != "null")  
 .groupBy("Parent_sys_id")
.agg(count("id").alias("tweets"))
.orderBy("tweets", ascending=False)
.limit(10)
 .show())




+-------------+------+
|Parent_sys_id|tweets|
+-------------+------+
|       681159|   554|
|       605727|   530|
|       603804|   518|
|       674825|   493|
|       460820|   235|
|       738975|   210|
|       758581|   154|
|       488086|   136|
|       461756|   125|
|       710916|   115|
+-------------+------+



In [16]:
#10 mensajes con mas respuestas
#Con spark.sql

masres=spark.sql("""
SELECT Parent_sys_id, count(Id) as `tweets` FROM tweets
WHERE (Tipe_action = "TW" OR Tipe_action = "MT") 
and Parent_sys_id !="sin padre" and Body != "null"
GROUP BY  Parent_sys_id
ORDER BY tweets DESC
LIMIT 10
""")
masres.show()


+-------------+------+
|Parent_sys_id|tweets|
+-------------+------+
|       681159|   554|
|       605727|   530|
|       603804|   518|
|       674825|   493|
|       460820|   235|
|       738975|   210|
|       758581|   154|
|       488086|   136|
|       461756|   125|
|       710916|   115|
+-------------+------+



## Ahora, restringe la búsqueda a los mensajes en el intervalo 2014-03-24 04:00 - 2014-03-24 10:00. (2.5 puntos).

In [17]:
#10 mensajes mas retweeteados en el intervalo de tiempo: Mensajes de typo RT y como los que 
#tienen padre=sin padre no estan entre los 10 primeros se puede poner o no. Se ha impuesto 
#solo en la version con sql. 

(events.select("Parent_sys_id","Id","Body","Pub_date")
.filter((events.Tipe_action == "RT") & (dayofmonth("Pub_date") == 24) &
 (hour("Pub_date") >= 4.0) & (hour("Pub_date") < 10.0) )
.groupBy("Parent_sys_id","Body")
.agg(count("id").alias("tweets"))
.orderBy("tweets", ascending=False)
.limit(10).show())



+-------------+--------------------+------+
|Parent_sys_id|                Body|tweets|
+-------------+--------------------+------+
|       645626|rt @jessansan: no...|   443|
|       603804|rt @marcmarquez93...|   429|
|       638320|rt @jessansan: qu...|   368|
|       645328|rt @jessansan: va...|   226|
|       605727|rt @lorenzo99: pe...|   161|
|       635916|rt @albitebaldi: ...|   140|
|       622993|rt @falcio78: @va...|   138|
|       553100|rt @motogp: #qata...|   109|
|       627447|rt @daniauryn: qu...|    89|
|       645225|rt @mflamigni: ch...|    89|
+-------------+--------------------+------+



In [18]:
#10 mas contestados

(events.select("Parent_sys_id","Id","Body","Pub_date")
.filter((events.Tipe_action == "TW") | (events.Tipe_action == "MT"))
.filter((events.Parent_sys_id != "sin padre") & (events.Body != "null") &  
 (dayofmonth("Pub_date") == 24) & (hour("Pub_date") >= 4) & (hour("Pub_date") < 10))
.groupBy("Parent_sys_id")
.agg(count("id").alias("tweets"))
.orderBy("tweets", ascending=False)
.limit(10)
.show())

+-------------+------+
|Parent_sys_id|tweets|
+-------------+------+
|       603804|    63|
|       605727|    46|
|       645328|    12|
|       645225|    12|
|       664891|     9|
|       645626|     9|
|       622993|     8|
|       626650|     7|
|       579001|     6|
|       645001|     4|
+-------------+------+



In [19]:
#10 mensajes mas retweeteados en intervalo de tiempo
#Con spark.sql

masret=spark.sql("""
SELECT Parent_sys_id, Body, count(Id) as `tweets` FROM tweets
WHERE Tipe_action = "RT"  and Parent_sys_id !="sin padre" and 
 DAYOFMONTH(Pub_date) = 24 AND HOUR(Pub_date) >= 4 AND HOUR(Pub_date) < 10
GROUP BY  Parent_sys_id, Body
ORDER BY tweets DESC
LIMIT 10
""")
masret.show()


+-------------+--------------------+------+
|Parent_sys_id|                Body|tweets|
+-------------+--------------------+------+
|       645626|rt @jessansan: no...|   443|
|       603804|rt @marcmarquez93...|   429|
|       638320|rt @jessansan: qu...|   368|
|       645328|rt @jessansan: va...|   226|
|       605727|rt @lorenzo99: pe...|   161|
|       635916|rt @albitebaldi: ...|   140|
|       622993|rt @falcio78: @va...|   138|
|       553100|rt @motogp: #qata...|   109|
|       627447|rt @daniauryn: qu...|    89|
|       645225|rt @mflamigni: ch...|    89|
+-------------+--------------------+------+



In [20]:
#10 mensajes con mas respuestas en intervalo de tiempo
#Con spark.sql

masres=spark.sql("""
SELECT Parent_sys_id, count(Id) as `tweets` FROM tweets
WHERE (Tipe_action = "TW" OR Tipe_action = "MT") and 
 DAYOFMONTH(Pub_date) = 24 AND HOUR(Pub_date) >= 4 AND HOUR(Pub_date) < 10
and Parent_sys_id !="sin padre" and Body != "null"
GROUP BY  Parent_sys_id
ORDER BY tweets DESC
LIMIT 10
""")
masres.show()


+-------------+------+
|Parent_sys_id|tweets|
+-------------+------+
|       603804|    63|
|       605727|    46|
|       645225|    12|
|       645328|    12|
|       645626|     9|
|       664891|     9|
|       622993|     8|
|       626650|     7|
|       579001|     6|
|       645001|     4|
+-------------+------+



In [21]:
# Remember to stop SparkContext before shutting down this notebook
sc.stop()